In [1]:
import numpy as np
import json
import pandas as pd
from tqdm import trange
import argparse
from modules.classical_multi import MultiVgg, MultiResNet
from utils.train_util import set_seed
from torch.utils.data import DataLoader
from datasets.dl import FNNData
from datasets.config import ModelConfig
import torch
import os
from tqdm import tqdm
import gc
import torch.nn.functional as F
from matplotlib import pyplot as plt

In [ ]:
input_data = torch.randint(low=0, high=256, size=(32, 6, 3, 224, 224))

In [ ]:
pic1 = input_data[0, 0].numpy()
print(pic1.shape)
img = pic1.transpose(1, 2, 0)
plt.imshow(img)
plt.show()

In [ ]:
input_data = input_data / 255.0

In [ ]:
# model = MultiVgg(ModelConfig())
model = MultiResNet(ModelConfig())

In [ ]:
res = model(input_data)

In [ ]:
res.shape

In [ ]:
input_label = torch.zeros((32), dtype=torch.long)

In [ ]:
loss = F.cross_entropy(res, input_label)

In [ ]:
loss

In [2]:
# train dataset
cfg = ModelConfig()
cfg.pic_matrix = np.random.randint(low=0, high=256, size=(1000, 3, 224, 224))
cfg.dataset = np.random.randint(low=0, high=1000, size=(128, 6))

In [5]:
train_dataset = FNNData(cfg)
train_data_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
input_label = torch.zeros((32), dtype=torch.long)

In [4]:
model = MultiResNet(ModelConfig())
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

In [6]:
for epoch in range(3):
    model.train()
    model.zero_grad()
    index = 0
    steps_one_epoch = len(train_data_loader)
    enum_dataloader = tqdm(train_data_loader, total=steps_one_epoch, desc="EP-{} train".format(epoch))
    for data in enum_dataloader:
    #     if index >= steps_one_epoch:
    #         break

        data = data / 256.0
        pred = model(data)
        loss = F.cross_entropy(pred, input_label)

        loss.backward()
        optimizer.step()
        model.zero_grad()

        enum_dataloader.set_description("EP-{} train loss: {}".format(epoch, loss))
        enum_dataloader.refresh()
        index += 1
    
    print('epoch {} end'.format(epoch))

EP-0 train loss: 2.0157570838928223: 100%|████████████████████████████████████████████████| 4/4 [02:26<00:00, 36.52s/it]


epoch 0 end


EP-1 train loss: 1.390627384185791: 100%|█████████████████████████████████████████████████| 4/4 [00:47<00:00, 11.91s/it]


epoch 1 end


EP-2 train loss: 1.5587198734283447: 100%|████████████████████████████████████████████████| 4/4 [01:55<00:00, 28.83s/it]

epoch 2 end


In [7]:
# test dataset
tcfg = ModelConfig()
tcfg.pic_matrix = np.random.randint(low=0, high=256, size=(1000, 3, 224, 224))
tcfg.dataset = np.random.randint(low=0, high=1000, size=(128, 11))

In [11]:
valid_dataset = FNNData(tcfg, isValidation=True)
valid_data_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

In [ ]:
model = MultiResNet(ModelConfig())

In [12]:
model.eval()  
with torch.no_grad():
    for data in valid_data_loader:
        print(data.size())
        input_data = data[:, :-10] / 256.0
        label_data = data[:, -10:]
        input_data = input_data.reshape(-1, 3, 224, 224)

        print(input_data.size(), label_data.size())

        res = model(input_data, test_mode=True)
        print(res.size())

torch.Size([32, 150538])
torch.Size([32, 3, 224, 224]) torch.Size([32, 10])
torch.Size([32, 100])
torch.Size([32, 150538])
torch.Size([32, 3, 224, 224]) torch.Size([32, 10])
torch.Size([32, 100])
torch.Size([32, 150538])
torch.Size([32, 3, 224, 224]) torch.Size([32, 10])
torch.Size([32, 100])
torch.Size([32, 150538])
torch.Size([32, 3, 224, 224]) torch.Size([32, 10])
torch.Size([32, 100])
